# More on Python packaging

Today we will learn more about packages and using git. Let's start by making a directory where we can do our work, and initialize it as a git repo.

If you haven't read [https://third-bit.com/py-rse/git-advanced.html](https://third-bit.com/py-rse/git-advanced.html), you should do that now.

The next cell simply starts us fresh. You *must* be very careful with `-fr`, it means to recursively delete the path you specify, and `f` means `force` which makes it work even when src doesn't exist. You can destroy a lot of work with this command.

In [4]:
%%bash
rm -fr src
pip uninstall -y s24pack

Next we use these commands to create a src directory with a package directory in it.



In [5]:
%%bash 
mkdir -p src/s24pack
cd src
git init
git checkout -b main
echo -e "s24 package\n===========" > README.md
git add README.md
git commit README.md -m "Initial readme."
git status

Initialized empty Git repository in /home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src/.git/


Switched to a new branch 'main'


[main (root-commit) cf8a00d] Initial readme.
 1 file changed, 2 insertions(+)
 create mode 100644 README.md
On branch main
nothing to commit, working tree clean


In [6]:
! tree src

src
├── README.md
└── s24pack

1 directory, 1 file


## Setting up our initial package



The next few cells create several files we talked about last time. We start with the setup.py file. You should edit this cell to replace <> fields with your information. This file references the license, and a script we will use as a command.



In [7]:
%%writefile src/setup.py
from setuptools import setup

setup(name='s24pack',
      version='0.0.1',
      description='s24 package',
      maintainer='John Kitchin',
      maintainer_email='jkitchin@cmu.edu',
      license='MIT',
      packages=['s24pack'],
      entry_points={'console_scripts': ['oa = s24pack.main:main']},
      long_description='''A long
      multiline description.''')

Writing src/setup.py


Next write the licence file.



In [8]:
%%writefile src/LICENSE
Copyright 2024 John Kitchin

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Writing src/LICENSE


The next three cells create the `__init__.py`, `utils.py`, and the script file.



In [9]:
%%writefile src/s24pack/__init__.py
print('loaded s24pack')
from .utils import hello
from .main import openalex_institution

Writing src/s24pack/__init__.py


In [10]:
%%writefile src/s24pack/utils.py
def hello(name):
    print(f'Hi there {name}')

Writing src/s24pack/utils.py


In [11]:
%%writefile src/s24pack/main.py
import click

import requests 
from collections.abc import Iterable 

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    if isinstance(query, str):
        query = '+'.join(query.split())

    # We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        
    url = f'https://api.openalex.org/institutions?search={query}'
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]

@click.command(help='OpenAlex Institutions')
@click.argument('query', nargs=-1)
def main(query):
    print('\n'.join(openalex_institution(query)))

Writing src/s24pack/main.py


In [12]:
!tree src

src
├── LICENSE
├── README.md
├── s24pack
│   ├── __init__.py
│   ├── main.py
│   └── utils.py
└── setup.py

1 directory, 6 files


In [14]:
import s24pack
s24pack.hello('test')

ModuleNotFoundError: No module named 's24pack'

In [15]:
import sys
sys.path.insert(0, 'src')

import s24pack
print(s24pack.__file__)

loaded s24pack
/home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src/s24pack/__init__.py


In [18]:
s24pack.hello('Class')
s24pack.openalex_institution('Carnegie Mellon University')

Hi there Class


['Carnegie Mellon University                            121925   4978056',
 'Carnegie Mellon University Qatar                         697      9769',
 'Carnegie Mellon University Australia                      93      2194',
 'Carnegie Mellon University Africa                        187      1179']

In [19]:
print(sys.path)
! pwd

['src', '/home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging', '/opt/tljh/user/lib/python39.zip', '/opt/tljh/user/lib/python3.9', '/opt/tljh/user/lib/python3.9/lib-dynload', '', '/home/jupyter-jkitchin@andrew.cm-11dd7/.local/lib/python3.9/site-packages', '/opt/tljh/user/lib/python3.9/site-packages']
/home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging


In [20]:
# Now let's rm src from the path
sys.path.remove('src')
sys.path

['/home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging',
 '/opt/tljh/user/lib/python39.zip',
 '/opt/tljh/user/lib/python3.9',
 '/opt/tljh/user/lib/python3.9/lib-dynload',
 '',
 '/home/jupyter-jkitchin@andrew.cm-11dd7/.local/lib/python3.9/site-packages',
 '/opt/tljh/user/lib/python3.9/site-packages']

# Installing the package

Let's go ahead and install this. Before we do that, a quick note about installation. There are system software packages, and you typically need elevated privileges to install those. You do not have them here. Instead, Python has a *user* space where you can install packages. In this JupyterHUB, you can find it here. Yours may look different because it depends on what you have installed.



To install our package we change into the src directory and run `pip install .` which means we run install in that directory.



In [21]:
! cd src && pip install .

Defaulting to user installation because normal site-packages is not writeable
Processing /home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src
  Preparing metadata (setup.py) ... done
  Created wheel for s24pack: filename=s24pack-0.0.1-py3-none-any.whl size=3095 sha256=08a3d5876e380f9af13c30d10c105f5d49a46d72f3829366db19942a61581c53
  Stored in directory: /tmp/pip-ephem-wheel-cache-ilcv279q/wheels/20/f0/20/5cf4991e8f7b80043f553076829214fc9222e851353c108bb8
Successfully built s24pack
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


The installation changed some things. First, It installed some packages in your local site packages. You can see there are some new s24pack directories.



Next, there are some changes in the src directory. There is a build directory, and an s24pack.egg-info directory.



In [22]:
!tree src

src
├── build
│   ├── bdist.linux-x86_64
│   └── lib
│       └── s24pack
│           ├── __init__.py
│           ├── main.py
│           └── utils.py
├── LICENSE
├── README.md
├── s24pack
│   ├── __init__.py
│   ├── main.py
│   ├── __pycache__
│   │   ├── __init__.cpython-39.pyc
│   │   ├── main.cpython-39.pyc
│   │   └── utils.cpython-39.pyc
│   └── utils.py
├── s24pack.egg-info
│   ├── dependency_links.txt
│   ├── entry_points.txt
│   ├── PKG-INFO
│   ├── SOURCES.txt
│   └── top_level.txt
└── setup.py

7 directories, 17 files


In [24]:
sys.path

['/home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging',
 '/opt/tljh/user/lib/python39.zip',
 '/opt/tljh/user/lib/python3.9',
 '/opt/tljh/user/lib/python3.9/lib-dynload',
 '',
 '/home/jupyter-jkitchin@andrew.cm-11dd7/.local/lib/python3.9/site-packages',
 '/opt/tljh/user/lib/python3.9/site-packages']

In [23]:
import s24pack
s24pack.__file__

'/home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src/s24pack/__init__.py'

In [26]:
import os
for path in sys.path:
    if os.path.exists(os.path.join(path, 's24pack')):
        print(f'Found in {path}')
        break

Found in /home/jupyter-jkitchin@andrew.cm-11dd7/.local/lib/python3.9/site-packages


In [27]:
s24pack.hello('John')

Hi there John


We have to switch to a terminal to check on our `oa` script. Try it out. I think the reason is the executable path in your terminal is different than the one here.



In [30]:
! echo $PATH
! which oa

/opt/tljh/user/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin


In [29]:
! oa carnegie mellon

/bin/bash: oa: command not found


There is a way to temporarily define the path at the command line here.

In [31]:
%%bash
PATH=/home/jupyter-jkitchin@andrew.cm-11dd7/.local/bin oa carnegie mellon

loaded s24pack
Carnegie Mellon University                            121925   4978056
Carnegie Mellon University Qatar                         697      9769
Carnegie Mellon University Australia                      93      2194
Carnegie Mellon University Africa                        187      1179


## Uninstall your package



In [32]:
! pip uninstall -y s24pack

Found existing installation: s24pack 0.0.1
Uninstalling s24pack-0.0.1:
  Successfully uninstalled s24pack-0.0.1


You can see here that the package is gone from site-packages now.



In [33]:
import os
for path in sys.path:
    if os.path.exists(os.path.join(path, 's24pack')):
        print(path)
        break

And you can see here the executable command is gone too.



In [34]:
! which oa

# Back to git

Before we reinstall, let's take some time to clean up our repo. Lets start with a high level view.



In [36]:
%%bash 
cd src
git status

On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	LICENSE
	build/
	s24pack.egg-info/
	s24pack/
	setup.py

nothing added to commit but untracked files present (use "git add" to track)


In [46]:
!tree src

src
├── build
│   ├── bdist.linux-x86_64
│   └── lib
│       └── s24pack
│           ├── __init__.py
│           ├── main.py
│           └── utils.py
├── LICENSE
├── README.md
├── s24pack
│   ├── __init__.py
│   ├── main.py
│   └── utils.py
├── s24pack.egg-info
│   ├── dependency_links.txt
│   ├── entry_points.txt
│   ├── PKG-INFO
│   ├── SOURCES.txt
│   └── top_level.txt
└── setup.py

6 directories, 14 files


In the src dir, we want to ignore a few things like the whole build dir, and the .egg-info directory. Lets make a .gitignore file first.



In [37]:
%%writefile src/.gitignore
build
*.egg-info
.ipynb_checkpoints
__pycache__

Writing src/.gitignore


In [38]:
%%bash 
cd src
git status

On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	LICENSE
	s24pack/
	setup.py

nothing added to commit but untracked files present (use "git add" to track)


Now it looks like we can just add everything and get going. After we add them, we check to see what is in there before we commit. Note we get a warning that files were ignored.



In [39]:
%%bash 
cd src
git add .gitignore *
git commit .gitignore -m "add ignore files"
git status

The following paths are ignored by one of your .gitignore files:
build
s24pack.egg-info
Use -f if you really want to add them.


[main ad619e2] add ignore files
 1 file changed, 4 insertions(+)
 create mode 100644 .gitignore
On branch main
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   LICENSE
	new file:   s24pack/__init__.py
	new file:   s24pack/main.py
	new file:   s24pack/utils.py
	new file:   setup.py



Now we commit these. 



In [40]:
%%bash 
cd src
git commit -m "First set of files"

[main 5c34fd1] First set of files
 5 files changed, 49 insertions(+)
 create mode 100644 LICENSE
 create mode 100644 s24pack/__init__.py
 create mode 100644 s24pack/main.py
 create mode 100644 s24pack/utils.py
 create mode 100644 setup.py


In [41]:
%%bash
cd src
git status
git log --graph --oneline

On branch main
nothing to commit, working tree clean
* 5c34fd1 First set of files
* ad619e2 add ignore files
* cf8a00d Initial readme.


Note there is a hash that we can use later, but it is hard to remember. Let's go ahead and add a tag to indicate we are at version 0.0.1. Technically this is a *lightweight* tag (https://git-scm.com/book/en/v2/Git-Basics-Tagging).



In [42]:
%%bash 
cd src
git tag v0.0.1

In [45]:
%%bash 
cd src
git status
git log 

On branch main
nothing to commit, working tree clean
commit 5c34fd1420997a33bd8e483f29bf5993a3110050
Author: Your Name <you@example.com>
Date:   Mon Mar 18 15:14:11 2024 +0000

    First set of files

commit ad619e2880f49c9b6705306fa47912d63b0f6217
Author: Your Name <you@example.com>
Date:   Mon Mar 18 15:13:07 2024 +0000

    add ignore files

commit cf8a00d438f2714e58ce7c297a0b1dbfce781f58
Author: Your Name <you@example.com>
Date:   Mon Mar 18 14:51:46 2024 +0000

    Initial readme.


# Let's catch our breath

1. We setup a small Python package with one executable script (oa), and one function in a utils.py file.
2. We installed it, and checked out what happened, where files were put, and that it worked.
3. We uninstalled, and checked if things got cleaned up.
4. We put the files under version control, and tagged v0.0.1

The package is currently uninstalled, and the repo should be clean. We are going to start making some changes now.

The `oa` script is not as reusable as we might like. The function in it does not need to be there. Let's move it to the utils.py file.  This requires us to change several files. In addition to moving the function, we have to move some imports, and modify the `__init__.py` file. Let's go ahead and do that.



In [46]:
%%writefile src/s24pack/utils.py 
import requests 
from collections.abc import Iterable 

def hello(name):
    print(f'Hi there {name}')
    

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    if isinstance(query, str):
        query = '+'.join(query.split())

    # We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        
    url = f'https://api.openalex.org/institutions?search={query}'
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]

Overwriting src/s24pack/utils.py


In [47]:
%%writefile src/s24pack/main.py 
import click
from .utils import openalex_institution

@click.command(help='OpenAlex Institutions')
@click.argument('query', nargs=-1)
def main(query):
    print('\n'.join(openalex_institution(query)))

Overwriting src/s24pack/main.py


In [48]:
%%writefile src/s24pack/__init__.py
from .utils import hello, openalex_institution

Overwriting src/s24pack/__init__.py


# Reinstall the package after making the changes.
You probably need to restart the kernel after this.



In [49]:
! cd src && pip install .

Defaulting to user installation because normal site-packages is not writeable
Processing /home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src
  Preparing metadata (setup.py) ... done
  Created wheel for s24pack: filename=s24pack-0.0.1-py3-none-any.whl size=3116 sha256=29b97758ffc9f75c5934cc0cc1708ea1005238bca777014e87161864df773ea7
  Stored in directory: /tmp/pip-ephem-wheel-cache-8a0p8f4s/wheels/20/f0/20/5cf4991e8f7b80043f553076829214fc9222e851353c108bb8
Successfully built s24pack
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [50]:
import s24pack
s24pack.hello('John')

Hi there John


In [51]:
# check that our function works.
s24pack.openalex_institution('carnegie+mellon')

['Carnegie Mellon University                            121925   4978056',
 'Carnegie Mellon University Qatar                         697      9769',
 'Carnegie Mellon University Australia                      93      2194',
 'Carnegie Mellon University Africa                        187      1179']

In [52]:
# Check that the command still works
! PATH=/home/jupyter-jkitchin@andrew.cm-11dd7/.local/bin oa carnegie mellon

Carnegie Mellon University                            121925   4978056
Carnegie Mellon University Qatar                         697      9769
Carnegie Mellon University Australia                      93      2194
Carnegie Mellon University Africa                        187      1179


## Commit changes to git when everything is working.

You can see there are some new nuisance files (check the git gui) we should ignore. Let's take care of that. You can either edit the .gitignore file, or run this cell.



In [53]:
! cd src && git status

On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   s24pack/__init__.py
	modified:   s24pack/main.py
	modified:   s24pack/utils.py

no changes added to commit (use "git add" and/or "git commit -a")


In [55]:
%%bash
echo -e "*checkpoint*" >> src/.gitignore

In [56]:
%%bash
cd src
git status

On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore
	modified:   s24pack/__init__.py
	modified:   s24pack/main.py
	modified:   s24pack/utils.py

no changes added to commit (use "git add" and/or "git commit -a")


Now, we can commit the results. It takes a little planning; I commit the .gitignore separately, since it is unrelated to the set of changes we make. Then, all that is left are the remaining files, so we commit them all at once. 



In [57]:
%%bash
cd src
git commit .gitignore -m "ignore checkpoint files"
git commit -am "move openalex_institutions function out of oa into utils.py"

[main aa7099a] ignore checkpoint files
 1 file changed, 1 insertion(+)
[main ee79aec] move openalex_institutions function out of oa into utils.py
 3 files changed, 12 insertions(+), 35 deletions(-)
 rewrite s24pack/main.py (79%)
 copy s24pack/{main.py => utils.py} (79%)


In [58]:
%%bash
cd src
git status

On branch main
nothing to commit, working tree clean


## Seeing older versions of files
We can see older versions of our files like this:



In [61]:
! cat src/s24pack/utils.py

import requests 
from collections.abc import Iterable 

def hello(name):
    print(f'Hi there {name}')
    

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    if isinstance(query, str):
        query = '+'.join(query.split())

    # We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        
    url = f'https://api.openalex.org/institutions?search={query}'
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]


In [59]:
%%bash
cd src
git show v0.0.1:s24pack/utils.py

def hello(name):
    print(f'Hi there {name}')


Compare that to our current version. HEAD always points to the most recent version.



In [ ]:
%%bash
cd src
git show HEAD:s24pack/utils.py

import requests 
from collections.abc import Iterable 

def hello(name):
    print(f'Hi there {name}')
    

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    if isinstance(query, str):
        query = '+'.join(query.split())

    # We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        query = '+'.join(query)
        
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]


In [62]:
%%bash
cd src
git diff v0.0.1:s24pack/utils.py HEAD:s24pack/utils.py

diff --git a/s24pack/utils.py b/s24pack/utils.py
index cda8cc4..582f959 100644
--- a/s24pack/utils.py
+++ b/s24pack/utils.py
@@ -1,2 +1,22 @@
+import requests 
+from collections.abc import Iterable 
+
 def hello(name):
     print(f'Hi there {name}')
+    
+
+def openalex_institution(query):
+    'query is a list of terms in the query, or a string.'
+    if isinstance(query, str):
+        query = '+'.join(query.split())
+
+    # We assume it is an iterable of strings.
+    elif isinstance(query, Iterable):
+        query = '+'.join(query)
+        
+    url = f'https://api.openalex.org/institutions?search={query}'
+    req = requests.get(url)
+    data = req.json()
+
+    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
+            for result in data['results']]


In [63]:
%%bash
cd src
git log --oneline

ee79aec move openalex_institutions function out of oa into utils.py
aa7099a ignore checkpoint files
5c34fd1 First set of files
ad619e2 add ignore files
cf8a00d Initial readme.


You can also use a hash to indicate which version you want to see. I use a relative notation here that means the second to last commit from HEAD.

In [64]:
%%bash
cd src
git show HEAD~2:s24pack/utils.py

def hello(name):
    print(f'Hi there {name}')


# Summary - take two
We have made our package a little better now. It still has the script, but it also has an importable function you can reuse in other applications, e.g. this notebook. There are a few things that pull this together:

1. setup.py has information about the package and script location for installing it.
2. utils.py has code that is imported in the oa script
3. `__init__.py` makes sure the function is imported and available

Leaving any of those details out makes something stop working.



# Testing

So far we have been testing by hand. That is moderately tedious... Every time we make changes, we have to go through and check if we broke something. We can set up some tests to help us with this.

Here is a simple test we can try.



In [65]:
%%writefile src/test_oa.py
import s24pack

def test_hello():
    assert s24pack.hello('John') == 'Hi there John'

Writing src/test_oa.py


We use [pytest](https://docs.pytest.org/en/7.2.x/contents.html) to run the test. You just run `pytest` at the command line.



In [68]:
%%bash
cd src
pytest

============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.2.2, pluggy-1.3.0
rootdir: /home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src
plugins: typeguard-2.13.3, anyio-3.6.1
collected 1 item

test_oa.py .                                                             [100%]

============================== 1 passed in 0.08s ===============================


Oh no! see if you can figure out the problem here. Fix the problem and commit the files to git. Note you will see some new files you should ignore in git.

The probem is that we only printed the result in `utils.hello`, and so the function returns None. You can fix it like this. It is moderately tedious that you have to reinstall the package after doing this. That is avoidable, but we save that for a later lesson.

In [67]:
%%writefile src/s24pack/utils.py 
import requests 
from collections.abc import Iterable 

def hello(name):
    return(f'Hi there {name}')
    

def openalex_institution(query):
    'query is a list of terms in the query, or a string.'
    if isinstance(query, str):
        query = '+'.join(query.split())

    # We assume it is an iterable of strings.
    elif isinstance(query, Iterable):
        query = '+'.join(query)
        
    url = f'https://api.openalex.org/institutions?search={query}'
    req = requests.get(url)
    data = req.json()

    return [f'{result["display_name"]:50s}{result["works_count"]:10d}{result["cited_by_count"]:10d}'
            for result in data['results']]

Overwriting src/s24pack/utils.py


In [69]:
%%bash
cd src
pip install . > /dev/null  # silent installation
pytest

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


============================= test session starts ==============================
platform linux -- Python 3.9.7, pytest-7.2.2, pluggy-1.3.0
rootdir: /home/jupyter-jkitchin@andrew.cm-11dd7/s24-06643/sse/04-more-python-packaging/src
plugins: typeguard-2.13.3, anyio-3.6.1
collected 1 item

test_oa.py .                                                             [100%]

============================== 1 passed in 0.08s ===============================


Re-read [https://third-bit.com/py-rse/scripting.html](https://third-bit.com/py-rse/scripting.html) on building python functions and scripts.

Then, read [https://third-bit.com/py-rse/packaging.html](https://third-bit.com/py-rse/packaging.html) about python packages. It is a little more involved than we have done so far, but you should be in good shape to read about it now. We do not use virtual environments here. I think they add a layer of complexity we don't want now, and there are many complications in using them (mostly in the form of what virtual environment am I in, and is it active).